In [1]:
from google.cloud import bigquery
from google.cloud import bigquery_connection_v1 as bq_connection
from google.cloud import bigquery_storage
from abc import ABC
from datetime import datetime
import google.auth
import pandas as pd
from google.cloud.exceptions import NotFound
from google.cloud import aiplatform
from vertexai.generative_models import GenerationConfig
import vertexai
import yaml
import asyncio
from agent import sqlagents
import tabulate
import google.generativeai as genai
import google.ai.generativelanguage as glm
from google.generativeai import caching
import re
import pickle
import json
import xgboost as xgb
from core.modelscorer import ModelScorer
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io
import base64
from agent import taskscheduler,oracle,VisualizeAgent
import numpy as np
import streamlit as st
from langchain_core.messages import AIMessage, HumanMessage
import uuid
from google.cloud import secretmanager
from contextlib import contextmanager, redirect_stdout
from io import StringIO
from typing import Dict, Text
import time
import ast

#load_dotenv()


def access_secret_version(project_id, secret_id, version_id="latest"):
    """
    Access the payload for the given secret version if one exists. The version
    can be a version number as a string (e.g. "5") or an alias (e.g. "latest").
    """
    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version.
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version.
    response = client.access_secret_version(name=name)

    # Return the secret payload.
    # WARNING: Do not print the secret in production.
    payload = response.payload.data.decode("UTF-8")
    return payload


#GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
project_id = "mlchatagent-429005"
secret_id = "GOOGLE_API_KEY"

# Access the secret
GOOGLE_API_KEY = access_secret_version(project_id, secret_id)


# Load configuration files
with open('sql_config.yml', 'r') as f:
    sql_config = yaml.load(f, Loader=yaml.FullLoader)

with open('conf_telchurn.yml', 'r') as f:
    model_config = yaml.load(f, Loader=yaml.FullLoader)

# Initialize Vertex AI and BigQuery
vertexai.init(project=sql_config['bigquery']['project_id'], location=sql_config['bigquery']['region'])
aiplatform.init(project=sql_config['bigquery']['project_id'], location=sql_config['bigquery']['region'])

# Initialize Agents and other components
Agent = sqlagents.Agent
embedder = sqlagents.EmbedderAgent('vertex')
SQLBuilder = sqlagents.BuildSQLAgent('gemini-1.5-flash-001')
SQLValidator = sqlagents.ValidateSQLAgent('gemini-1.5-flash-001')
ResponseAgent = sqlagents.ResponseAgent('gemini-1.5-flash-001')
SQLDebugger = sqlagents.DebugSQLAgent('gemini-1.5-flash-001')
QueryRefiller=sqlagents.QueryRefiller('gemini-1.5-flash-001')



bq_connector = sqlagents.BQConnector(
    project_id=sql_config['bigquery']['project_id'],
    dataset_name=sql_config['bigquery']['dataset_id'],
    region=sql_config['bigquery']['region']
)

task_master = taskscheduler.TaskMaster()
churn_explainer = oracle.ShapOracle()
xgb_scorer = ModelScorer()
visualize_agent = VisualizeAgent.VisualizeAgent()

# Constants from the configuration file
USER_DATABASE = sql_config['bigquery']['dataset_id']
call_await = sql_config['sql_run']['call_await']
num_table_matches = sql_config['sql_run']['num_table_matches']
num_column_matches = sql_config['sql_run']['num_column_matches']
table_similarity_threshold = sql_config['sql_run']['table_similarity_threshold']
column_similarity_threshold = sql_config['sql_run']['column_similarity_threshold']
example_similarity_threshold = sql_config['sql_run']['example_similarity_threshold']
num_sql_matches = sql_config['sql_run']['num_sql_matches']
DEBUGGING_ROUNDS = sql_config['sql_run']['DEBUGGING_ROUNDS']
RUN_DEBUGGER = sql_config['sql_run']['RUN_DEBUGGER']
LLM_VALIDATION = sql_config['sql_run']['LLM_VALIDATION']
EXECUTE_FINAL_SQL = sql_config['sql_run']['EXECUTE_FINAL_SQL']
VECTOR_STORE = sql_config['sql_run']['VECTOR_STORE']
DATA_SOURCE = sql_config['sql_run']['DATA_SOURCE']
KGQ_ENABLED = sql_config['sql_run']['KGQ_ENABLED']
with open(model_config['model']['shap_base_value'], "r") as file:
    base_value = float(file.read().strip())


def remove_sql_and_backticks(input_text):
    modified_text = re.sub(r'```|sql', '', input_text)
    modified_text = re.sub(r'\\\\', '', modified_text)
    return modified_text

def normalize_string(s):
    s = s.lower()
    s = re.sub(r'\s+', ' ', s)  # Replace multiple spaces with a single space
    s = re.sub(r'[^a-zA-Z0-9\s]', '', s)  # Remove all non-alphanumeric characters except spaces
    return s.strip()

d:\Work\Github\mlchatagent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
def subset_shap_summary(customer_data_sql_query:str,shap_data_sql_query:str,user_question:str):
    """
        Calculates the SHAP summary of customers from the customer data query and SHAP feature contribution data for same subset.
        This can be used to identify patterns, top churn contributors and feature importance for the subset of data.
        Use this tool always for understanding the main reasons for churn for a subset of customers.
        Use this tool for identifying top churn contributors customers for any subset of data.
        Use this tool to identify main reasons for churn for a subset of customers user wants to analyze.


        Parameters:
        ----------
        customer_data_sql_query : str
            The SQL query to retrieve the customer data for a subset of customers as asked by user.
            Should return all columns from customer data
            Use select * in the query. Don't limit the columns in the query. 
        shap_data_sql_query : str
            The SQL query to retrieve the SHAP feature contribution data for the same subset of customers as asked by user. 
            Should return all columns from shap value data
            Use select * in the query. Don't limit the columns in the query. 
        user_question:str
            The user's question that the SQL query is intended to answer.

        Returns:
        -------
        str
            A report on reasons for churn for the user question. You have to display this report to the user.
        
        Notes
        -----
        - It is important to have both the customer data query and SHAP data query for the same subset of customers to be passed as input.
        - The output from this is the report. You have to display this report to the user as it is. DO NOT MODIFY THE OUTPUT.
        - Add a final note after the report of how nd why the recommended actions should be tested with churn adn clv impact analysis.
"""
    st.markdown("--------------------------------------🔍 *Subset Churn Analysis Tool* 🔍--------------------------------------")

    try:
        customer_data_sql_query=remove_sql_and_backticks(customer_data_sql_query)
        customer_data_sql_query=customer_data_sql_query.replace("\\", "")
        shap_data_sql_query=remove_sql_and_backticks(shap_data_sql_query)
        shap_data_sql_query=shap_data_sql_query.replace("\\", "")


        ##Get the subset data from bigquery
        df_data=bq_connector.retrieve_df(customer_data_sql_query)
        df_shap_data=bq_connector.retrieve_df(shap_data_sql_query)

        ##Order both dataframes by customerid
        df_data=df_data.sort_values(by='customerid')
        df_shap_data=df_shap_data.sort_values(by='customerid')

        df_shap_data=pd.merge(df_shap_data,df_data[['customerid']],on='customerid',how='inner')

        def sigmoid(x):
            """ Sigmoid function to convert log-odds to probabilities. """
            return 1 / (1 + np.exp(-x))
        
        print(base_value)
        base_probability = sigmoid(base_value)
        results = []
        feature_importances = {}
        # Process each feature
        ##Remove the SHAP prefix from SHAP data columns
        df_shap_data.columns = df_shap_data.columns.str.replace('shapvalue_', '')
        common_columns = df_data.columns.intersection(df_shap_data.columns)
        #Remove if column customerid exists
        columns_to_drop = ['customerid', 'churn']

        for col in columns_to_drop:
            if col in common_columns:
                common_columns = common_columns.drop(col)

        print(f"data shape:{df_data.shape}")
        print(f"shap data shape:{df_shap_data.shape}")
        # Calculate feature importances
        for feature in common_columns:
            feature_shap_values = df_shap_data[feature]
            feature_importances[feature] = np.mean(np.abs(feature_shap_values))


        importance_df = pd.DataFrame(list(feature_importances.items()), columns=['Feature', 'Importance'])
        print(importance_df.head())
        importance_df.sort_values('Importance', ascending=False, inplace=True)
        importance_df['Rank'] = range(1, len(importance_df) + 1)
        importance_ranks = importance_df.set_index('Feature')['Rank'].to_dict()
        #print(2)

        if df_data.shape[0]>10000:
            subset_levels=5
        elif df_data.shape[0]>5000:
            subset_levels=5
        elif df_data.shape[0]>1000:
            subset_levels=3
        else:
            subset_levels=2
        print(subset_levels)

        df_shap_data=df_shap_data.reset_index(drop=True)
        df_data=df_data.reset_index(drop=True)
        
        for feature in common_columns:
            feature_values = df_data[feature]
            feature_shap_values = df_shap_data[feature]
            df = pd.DataFrame({feature: feature_values, 'SHAP Value': feature_shap_values})
            numeric_features = df_data.select_dtypes(include=['number']).columns

            
            if feature in numeric_features:
                # Equal sized bins take out SHAP patterns as we get in SHAP PDP
                #df['Group'] = pd.qcut(df[feature], subset_levels, duplicates='drop')
                # Equal length buckets are used instead
                min_val = df[feature].min()
                max_val = df[feature].max()
                range_width = max_val - min_val
                bin_edges = np.linspace(min_val - 0.01 * range_width, max_val + 0.01 * range_width, subset_levels + 1)
                #print(feature)
                df['Group'] = pd.cut(df[feature], subset_levels)
            else:
                df['Group'] = df[feature]

            
            group_avg = df.groupby('Group', observed=True).agg({
                'SHAP Value': 'mean',
                feature: 'count'
            }).reset_index()

            group_avg.rename(columns={feature: 'Count'}, inplace=True)
            group_avg['Adjusted Probability'] = sigmoid(base_value + group_avg['SHAP Value'])
            group_avg['Probability Change (%)'] = (group_avg['Adjusted Probability'] - base_probability) * 100
            group_avg['Feature'] = feature
            group_avg['Feature Importance'] = feature_importances[feature]
            group_avg['Importance Rank'] = importance_ranks[feature]
            results.append(group_avg)

            if feature == 'currentequipmentdays':
                chk = df
                chk['customerid'] = df_data['customerid']    
                chk2 = group_avg
        result_df = pd.concat(results, ignore_index=True)
        print(result_df.shape)

        if df_data.shape[0] > 1000:
            ##Count of groups should be atleast 50 records - If not remove the group
            result_df = result_df[result_df['Count'] >= 50]
            result_df.sort_values(['Importance Rank','SHAP Value', 'Probability Change (%)'], ascending=[True,False, False], inplace=True)
               
        copy=result_df[['Feature','Group','SHAP Value','Adjusted Probability','Probability Change (%)','Feature Importance','Importance Rank','Count']].copy()
        # result_df=result_df[result_df['Importance Rank']<=10]
        #result_df = result_df.head(100)
        #print(tabulate.tabulate(result_df[['Feature','Group','Probability Change (%)','SHAP Value','Importance Rank']].head(30), headers='keys', tablefmt='pipe', showindex='never'))
        ##Limit Data with positive SHAP contributions only
        result_df = result_df[result_df['SHAP Value'] > 0]
        result_df=result_df[result_df['Importance Rank']<=25]
        print(result_df.shape)



        return copy,result_df,chk,chk2,df_shap_data
    except Exception as e:
        return str(e)

In [39]:
q1="SELECT * FROM `mlchatagent-429005.telecom_churn.customer_data`"
q2="SELECT * FROM `mlchatagent-429005.telecom_churn.customer_shap_data`"

r1,r2,chk,chk2,df_shap_data=subset_shap_summary(q1,q2,"Top churn contributors for the subset of customers")

-0.9089516997337341
data shape:(51047, 63)
shap data shape:(51047, 61)
              Feature  Importance
0        childreninhh    0.030679
1  handsetrefurbished    0.051141
2   handsetwebcapable    0.029795
3          truckowner    0.001248
4             rvowner    0.001018
5
(296, 8)
(52, 8)


In [40]:
r1[r1['Feature']=='currentequipmentdays']

,Feature,Group,SHAP Value,Adjusted Probability,Probability Change (%),Feature Importance,Importance Rank,Count
238,currentequipmentdays,"(1085.2, 1448.6]",0.560993,0.413877,12.666308,0.29302,1,650
239,currentequipmentdays,"(1448.6, 1812.0]",0.526154,0.405452,11.823800,0.29302,1,100
237,currentequipmentdays,"(721.8, 1085.2]",0.348459,0.363434,7.621917,0.29302,1,4611
236,currentequipmentdays,"(358.4, 721.8]",0.188709,0.327340,4.012511,0.29302,1,17395
235,currentequipmentdays,"(-6.817, 358.4]",-0.253884,0.238152,-4.906192,0.29302,1,28291


In [41]:
chk2

,Group,SHAP Value,Count,Adjusted Probability,Probability Change (%),Feature,Feature Importance,Importance Rank
0,"(-6.817, 358.4]",-0.253884,28291,0.238152,-4.906192,currentequipmentdays,0.29302,1
1,"(358.4, 721.8]",0.188709,17395,0.327340,4.012511,currentequipmentdays,0.29302,1
2,"(721.8, 1085.2]",0.348459,4611,0.363434,7.621917,currentequipmentdays,0.29302,1
3,"(1085.2, 1448.6]",0.560993,650,0.413877,12.666308,currentequipmentdays,0.29302,1
4,"(1448.6, 1812.0]",0.526154,100,0.405452,11.823800,currentequipmentdays,0.29302,1


In [42]:
chk

,currentequipmentdays,SHAP Value,Group,customerid
0,361.0,0.224565,"(358.4, 721.8]",3000002
1,1504.0,0.513336,"(1448.6, 1812.0]",3000010
2,1812.0,0.519439,"(1448.6, 1812.0]",3000014
3,458.0,0.169947,"(358.4, 721.8]",3000022
4,852.0,0.212443,"(721.8, 1085.2]",3000026
...,...,...,...,...
51042,526.0,0.215769,"(358.4, 721.8]",3399958
51043,464.0,0.187165,"(358.4, 721.8]",3399974
51044,378.0,0.271913,"(358.4, 721.8]",3399978
51045,433.0,0.310654,"(358.4, 721.8]",3399990


In [28]:
df_shap_data[['customerid','currentequipmentdays']].head(10)

,customerid,currentequipmentdays
0,3000002,0.224565
1,3000010,0.513336
2,3000014,0.519439
3,3000022,0.169947
4,3000026,0.212443
5,3000030,-0.343521
6,3000038,0.213118
7,3000042,0.204588
8,3000046,0.212450
9,3000050,0.201909


In [32]:
df_shap_data[df_shap_data['customerid']==3049902]['currentequipmentdays']

6366    0.281751
Name: currentequipmentdays, dtype: float64